In [6]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

def get_image_paths(folder):
    img_dir = os.path.join(folder, "image")
    return [os.path.join(img_dir, fname) for fname in os.listdir(img_dir) if fname.endswith('.png')]

all_image_paths = []
for split in ["train", "val", "test"]:
    all_image_paths.extend(get_image_paths(split))

means = []
stds = []

for img_path in tqdm(all_image_paths, desc="Calculating mean/std"):
    img = Image.open(img_path).convert("RGB")
    img = np.array(img) / 255.0  # scale to [0,1]
    means.append(np.mean(img, axis=(0,1)))
    stds.append(np.std(img, axis=(0,1)))

mean = np.mean(means, axis=0)
std = np.mean(stds, axis=0)

print(f"Dataset mean: {mean}")
print(f"Dataset std: {std}")

Calculating mean/std: 100%|██████████| 300/300 [00:01<00:00, 267.74it/s]

Dataset mean: [0.11803523 0.12175034 0.12110489]
Dataset std: [0.16351671 0.16600904 0.16600859]


U-Net

In [21]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# -----------------------
# Dataset
# -----------------------
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])  # assumes same filename

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # grayscale

        if self.transform is not None:
            image = self.transform(image)
            # For mask: only resize and ToTensor, no Normalize!
            mask_transform = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.ToTensor()
            ])
            mask = mask_transform(mask)

        mask = (mask > 0).float()  # binarize
        return image, mask

# -----------------------
# U-Net
# -----------------------
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        self.enc1 = DoubleConv(in_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)

        self.pool = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(128, 64)

        self.conv_final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.upconv4(b)
        d4 = torch.cat((d4, e4), dim=1)
        d4 = self.dec4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((d3, e3), dim=1)
        d3 = self.dec3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)

        return torch.sigmoid(self.conv_final(d1))

# -----------------------
# Metrics
# -----------------------
def dice_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    return ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)).item()

def iou_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return ((intersection + smooth) / (union + smooth)).item()

def pixel_accuracy(pred, target):
    pred = (pred > 0.5).float()
    correct = (pred == target).float().sum()
    return (correct / target.numel()).item()

def precision_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    tp = (pred * target).sum()
    fp = (pred * (1 - target)).sum()
    return (tp / (tp + fp + smooth)).item()

def recall_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    tp = (pred * target).sum()
    fn = ((1 - pred) * target).sum()
    return (tp / (tp + fn + smooth)).item()

# -----------------------
# Data Setup
# -----------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.11803523, 0.12175034, 0.12110489], std=[0.16351671, 0.16600904, 0.16600859])
])

train_dataset = SegmentationDataset(
    img_dir="train/image",
    mask_dir="train/mask",
    transform=transform
)
val_dataset = SegmentationDataset(
    img_dir="val/image",
    mask_dir="val/mask",
    transform=transform
)
test_dataset = SegmentationDataset(
    img_dir="test/image",
    mask_dir="test/mask",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

# -----------------------
# U-Net Training
# -----------------------
model = UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
bce = nn.BCELoss()

num_epochs = 20
u_net_best_test_loss = float("inf")
patience = 5
epochs_no_improve = 0

for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    u_net_train_loss, u_net_train_dice, u_net_train_iou, u_net_train_acc = 0, 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)
        preds = model(imgs)
        loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        u_net_train_loss += loss.item()
        u_net_train_dice += dice_score(preds, masks)
        u_net_train_iou += iou_score(preds, masks)
        u_net_train_acc += pixel_accuracy(preds, masks)

    # --- Validation ---
    model.eval()
    u_net_val_loss, u_net_val_dice, u_net_val_iou, u_net_val_acc, u_net_val_prec, u_net_val_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            u_net_val_loss += loss.item()
            u_net_val_dice += dice_score(preds, masks)
            u_net_val_iou += iou_score(preds, masks)
            u_net_val_acc += pixel_accuracy(preds, masks)
            u_net_val_prec += precision_score(preds, masks)
            u_net_val_rec += recall_score(preds, masks)

    # --- Test ---
    u_net_test_loss, u_net_test_dice, u_net_test_iou, u_net_test_acc, u_net_test_prec, u_net_test_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Test"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            u_net_test_loss += loss.item()
            u_net_test_dice += dice_score(preds, masks)
            u_net_test_iou += iou_score(preds, masks)
            u_net_test_acc += pixel_accuracy(preds, masks)
            u_net_test_prec += precision_score(preds, masks)
            u_net_test_rec += recall_score(preds, masks)

    # --- Print metrics ---
    print(f"\nEpoch {epoch+1}:")
    print(f" Train Loss={u_net_train_loss/len(train_loader):.4f} | "
          f"Dice={u_net_train_dice/len(train_loader):.4f} | IoU={u_net_train_iou/len(train_loader):.4f} | Acc={u_net_train_acc/len(train_loader):.4f}")
    print(f" Val   Loss={u_net_val_loss/len(val_loader):.4f} | "
          f"Dice={u_net_val_dice/len(val_loader):.4f} | IoU={u_net_val_iou/len(val_loader):.4f} | Acc={u_net_val_acc/len(val_loader):.4f} | "
          f"Prec={u_net_val_prec/len(val_loader):.4f} | Recall={u_net_val_rec/len(val_loader):.4f}")
    print(f" Test  Loss={u_net_test_loss/len(test_loader):.4f} | "
          f"Dice={u_net_test_dice/len(test_loader):.4f} | IoU={u_net_test_iou/len(test_loader):.4f} | Acc={u_net_test_acc/len(test_loader):.4f} | "
          f"Prec={u_net_test_prec/len(test_loader):.4f} | Recall={u_net_test_rec/len(test_loader):.4f}")

    # --- Early stopping ---
    if u_net_test_loss < u_net_best_test_loss:
        u_net_best_test_loss = u_net_test_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_unet.pth")
        print("✅ Model saved!")
    else:
        epochs_no_improve += 1
        print(f"EarlyStopping counter: {epochs_no_improve} of {patience}")
        if epochs_no_improve >= patience:
            print("⏹️ Early stopping triggered!")
            break


Epoch 1/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 12.78it/s]



Epoch 1:
 Train Loss=0.8455 | Dice=0.0116 | IoU=0.0058 | Acc=0.5501
 Val   Loss=0.8042 | Dice=0.0267 | IoU=0.0135 | Acc=0.7578 | Prec=0.0136 | Recall=0.7947
 Test  Loss=0.8015 | Dice=0.0311 | IoU=0.0158 | Acc=0.7628 | Prec=0.0159 | Recall=0.8274
✅ Model saved!


Epoch 2/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 13.27it/s]



Epoch 2:
 Train Loss=0.7431 | Dice=0.0012 | IoU=0.0006 | Acc=0.9891
 Val   Loss=0.7396 | Dice=0.0010 | IoU=0.0005 | Acc=0.9930 | Prec=0.0015 | Recall=0.0007
 Test  Loss=0.7390 | Dice=0.0018 | IoU=0.0009 | Acc=0.9926 | Prec=0.0026 | Recall=0.0013
✅ Model saved!


Epoch 3/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 12.98it/s]


...


Epoch 18:
 Train Loss=0.1839 | Dice=0.8587 | IoU=0.7533 | Acc=0.9988
 Val   Loss=0.1898 | Dice=0.8444 | IoU=0.7311 | Acc=0.9987 | Prec=0.8340 | Recall=0.8559
 Test  Loss=0.2084 | Dice=0.8078 | IoU=0.6812 | Acc=0.9982 | Prec=0.7970 | Recall=0.8294
✅ Model saved!


Epoch 19/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 12.78it/s]



Epoch 19:
 Train Loss=0.1728 | Dice=0.8730 | IoU=0.7750 | Acc=0.9989
 Val   Loss=0.1865 | Dice=0.8411 | IoU=0.7265 | Acc=0.9987 | Prec=0.8250 | Recall=0.8605
 Test  Loss=0.2051 | Dice=0.8043 | IoU=0.6760 | Acc=0.9982 | Prec=0.7931 | Recall=0.8321
✅ Model saved!


Epoch 20/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 12.10it/s]



Epoch 20:
 Train Loss=0.1679 | Dice=0.8752 | IoU=0.7784 | Acc=0.9989
 Val   Loss=0.1847 | Dice=0.8379 | IoU=0.7218 | Acc=0.9987 | Prec=0.8572 | Recall=0.8214
 Test  Loss=0.2020 | Dice=0.8038 | IoU=0.6760 | Acc=0.9983 | Prec=0.8283 | Recall=0.7907
✅ Model saved!


In [ ]:
print(f"Best Test Loss: {u_net_test_loss/len(test_loader):.4f} at epoch {epoch + 1 - epochs_no_improve}")
print(f"Best Test Dice: {u_net_test_dice/len(test_loader):.4f}")
print(f"Best Test IoU: {u_net_test_iou/len(test_loader):.4f}")
print(f"Best Test Accuracy: {u_net_test_acc/len(test_loader):.4f}")
print(f"Best Test Precision: {u_net_test_prec/len(test_loader):.4f}")
print(f"Best Test Recall: {u_net_test_rec/len(test_loader):.4f}")

Best Test Loss: 0.2020 at epoch 17
Best Test Dice: 0.8038
Best Test IoU: 0.6760
Best Test Accuracy: 0.9983
Best Test Precision: 0.8283
Best Test Recall: 0.7907


In [57]:
u_net_loss = u_net_test_loss/len(test_loader)
u_net_dice = u_net_test_dice/len(test_loader)
u_net_iou = u_net_test_iou/len(test_loader)

In [25]:
import torchvision
from torchvision.models.segmentation import deeplabv3_resnet50

# -----------------------
# DeepLabV3 Model
# -----------------------
class DeepLabV3(nn.Module):
    def __init__(self, num_classes=1):
        super(DeepLabV3, self).__init__()
        self.model = deeplabv3_resnet50(weights=None, num_classes=1)
        # If you want to use pretrained backbone:
        self.model = deeplabv3_resnet50(weights="DEFAULT")
        self.model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, x):
        out = self.model(x)['out']
        return torch.sigmoid(out)

# -----------------------
# Training DeepLabV3
# -----------------------
deeplab_model = DeepLabV3().to(device)
optimizer = optim.Adam(deeplab_model.parameters(), lr=1e-4)
bce = nn.BCELoss()
num_epochs = 20
deep_lab_best_test_loss = float("inf")
patience = 5
epochs_no_improve = 0

for epoch in range(num_epochs):
    # --- Training ---
    deeplab_model.train()
    deep_lab_train_loss, deep_lab_train_dice, deep_lab_train_iou, deep_lab_train_acc = 0, 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f"DeepLabV3 Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)
        preds = deeplab_model(imgs)
        loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        deep_lab_train_loss += loss.item()
        deep_lab_train_dice += dice_score(preds, masks)
        deep_lab_train_iou += iou_score(preds, masks)
        deep_lab_train_acc += pixel_accuracy(preds, masks)

    # --- Validation ---
    deeplab_model.eval()
    deep_lab_val_loss, deep_lab_val_dice, deep_lab_val_iou, deep_lab_val_acc, deep_lab_val_prec, deep_lab_val_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"DeepLabV3 Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = deeplab_model(imgs)
            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            deep_lab_val_loss += loss.item()
            deep_lab_val_dice += dice_score(preds, masks)
            deep_lab_val_iou += iou_score(preds, masks)
            deep_lab_val_acc += pixel_accuracy(preds, masks)
            deep_lab_val_prec += precision_score(preds, masks)
            deep_lab_val_rec += recall_score(preds, masks)

    # --- Test ---
    deep_lab_test_loss, deep_lab_test_dice, deep_lab_test_iou, deep_lab_test_acc, deep_lab_test_prec, deep_lab_test_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(test_loader, desc=f"DeepLabV3 Epoch {epoch+1}/{num_epochs} - Test"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = deeplab_model(imgs)
            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            deep_lab_test_loss += loss.item()
            deep_lab_test_dice += dice_score(preds, masks)
            deep_lab_test_iou += iou_score(preds, masks)
            deep_lab_test_acc += pixel_accuracy(preds, masks)
            deep_lab_test_prec += precision_score(preds, masks)
            deep_lab_test_rec += recall_score(preds, masks)

    # --- Print metrics ---
    print(f"\nDeepLabV3 Epoch {epoch+1}:")
    print(f" Train Loss={deep_lab_train_loss/len(train_loader):.4f} | "
          f"Dice={deep_lab_train_dice/len(train_loader):.4f} | IoU={deep_lab_train_iou/len(train_loader):.4f} | Acc={deep_lab_train_acc/len(train_loader):.4f}")
    print(f" Val   Loss={deep_lab_val_loss/len(val_loader):.4f} | "
          f"Dice={deep_lab_val_dice/len(val_loader):.4f} | IoU={deep_lab_val_iou/len(val_loader):.4f} | Acc={deep_lab_val_acc/len(val_loader):.4f} | "
          f"Prec={deep_lab_val_prec/len(val_loader):.4f} | Recall={deep_lab_val_rec/len(val_loader):.4f}")
    print(f" Test  Loss={deep_lab_test_loss/len(test_loader):.4f} | "
          f"Dice={deep_lab_test_dice/len(test_loader):.4f} | IoU={deep_lab_test_iou/len(test_loader):.4f} | Acc={deep_lab_test_acc/len(test_loader):.4f} | "
          f"Prec={deep_lab_test_prec/len(test_loader):.4f} | Recall={deep_lab_test_rec/len(test_loader):.4f}")

    # --- Early stopping ---
    if deep_lab_test_loss < deep_lab_best_test_loss:
        deep_lab_best_test_loss = deep_lab_test_loss
        epochs_no_improve = 0
        torch.save(deeplab_model.state_dict(), "best_deeplabv3.pth")
        print("✅ DeepLabV3 Model saved!")
    else:
        epochs_no_improve += 1
        print(f"EarlyStopping counter: {epochs_no_improve} of {patience}")
        if epochs_no_improve >= patience:
            print("⏹️ Early stopping triggered!")
            break

DeepLabV3 Epoch 1/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 16.07it/s]



DeepLabV3 Epoch 1:
 Train Loss=0.8097 | Dice=0.0137 | IoU=0.0070 | Acc=0.7282
 Val   Loss=0.9446 | Dice=0.0000 | IoU=0.0000 | Acc=0.9692 | Prec=0.0000 | Recall=0.0000
 Test  Loss=0.9199 | Dice=0.0000 | IoU=0.0000 | Acc=0.9721 | Prec=0.0000 | Recall=0.0000
✅ DeepLabV3 Model saved!


DeepLabV3 Epoch 2/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 15.68it/s]



DeepLabV3 Epoch 2:
 Train Loss=0.7111 | Dice=0.0000 | IoU=0.0000 | Acc=0.9921
 Val   Loss=0.7000 | Dice=0.0000 | IoU=0.0000 | Acc=0.9901 | Prec=0.0000 | Recall=0.0000
 Test  Loss=0.6968 | Dice=0.0000 | IoU=0.0000 | Acc=0.9902 | Prec=0.0000 | Recall=0.0000
✅ DeepLabV3 Model saved!


DeepLabV3 Epoch 3/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 16.26it/s]


...


DeepLabV3 Epoch 18:
 Train Loss=0.1403 | Dice=0.7963 | IoU=0.6626 | Acc=0.9983
 Val   Loss=0.1980 | Dice=0.6825 | IoU=0.5345 | Acc=0.9974 | Prec=0.6919 | Recall=0.6827
 Test  Loss=0.2250 | Dice=0.6322 | IoU=0.4905 | Acc=0.9967 | Prec=0.6693 | Recall=0.6423
EarlyStopping counter: 3 of 5


DeepLabV3 Epoch 19/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 15.91it/s]



DeepLabV3 Epoch 19:
 Train Loss=0.1366 | Dice=0.7983 | IoU=0.6660 | Acc=0.9983
 Val   Loss=0.1558 | Dice=0.7560 | IoU=0.6137 | Acc=0.9979 | Prec=0.7152 | Recall=0.8070
 Test  Loss=0.1818 | Dice=0.7058 | IoU=0.5563 | Acc=0.9973 | Prec=0.6817 | Recall=0.7500
✅ DeepLabV3 Model saved!


DeepLabV3 Epoch 20/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 15.85it/s]


DeepLabV3 Epoch 20:
 Train Loss=0.1326 | Dice=0.8019 | IoU=0.6702 | Acc=0.9983
 Val   Loss=0.1730 | Dice=0.7204 | IoU=0.5691 | Acc=0.9977 | Prec=0.7072 | Recall=0.7386
 Test  Loss=0.2017 | Dice=0.6656 | IoU=0.5171 | Acc=0.9970 | Prec=0.6669 | Recall=0.6928
EarlyStopping counter: 1 of 5


In [26]:
print(f"Best Test Loss: {deep_lab_test_loss/len(test_loader):.4f} at epoch {epoch + 1 - epochs_no_improve}")
print(f"Best Test Dice: {deep_lab_test_dice/len(test_loader):.4f}")
print(f"Best Test IoU: {deep_lab_test_iou/len(test_loader):.4f}")
print(f"Best Test Accuracy: {deep_lab_test_acc/len(test_loader):.4f}")
print(f"Best Test Precision: {deep_lab_test_prec/len(test_loader):.4f}")
print(f"Best Test Recall: {deep_lab_test_rec/len(test_loader):.4f}")

Best Test Loss: 0.2017 at epoch 19
Best Test Dice: 0.6656
Best Test IoU: 0.5171
Best Test Accuracy: 0.9970
Best Test Precision: 0.6669
Best Test Recall: 0.6928


In [58]:
deep_lab_loss = deep_lab_test_loss/len(test_loader)
deep_lab_dice = deep_lab_test_dice/len(test_loader)
deep_lab_iou = deep_lab_test_iou/len(test_loader)

In [31]:
import torchvision
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

# -----------------------
# FPN Model
# -----------------------
class FPNHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(FPNHead, self).__init__()
        self.conv = nn.Conv2d(in_channels, 128, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.out = nn.Conv2d(128, num_classes, 1)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.out(x)
        return x

class FPNNet(nn.Module):
    def __init__(self, num_classes=1):
        super(FPNNet, self).__init__()
        # Use a pre-trained ResNet50 FPN backbone
        self.backbone = resnet_fpn_backbone('resnet50', pretrained=True)
        self.head = FPNHead(256, num_classes)  # 256 is the FPN out_channels

    def forward(self, x):
        features = self.backbone(x)
        # Use the highest resolution FPN output ('0')
        x = features['0']
        x = self.head(x)
        x = nn.functional.interpolate(x, size=(256, 256), mode='bilinear', align_corners=False)
        return torch.sigmoid(x)

# -----------------------
# Training FPN
# -----------------------
fpn_model = FPNNet().to(device)
optimizer = optim.Adam(fpn_model.parameters(), lr=1e-4)
bce = nn.BCELoss()
num_epochs = 20
best_test_loss = float("inf")
patience = 5
epochs_no_improve = 0

for epoch in range(num_epochs):
    # --- Training ---
    fpn_model.train()
    fpn_train_loss, fpn_train_dice, fpn_train_iou, fpn_train_acc = 0, 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f"FPN Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)
        preds = fpn_model(imgs)
        loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        fpn_train_loss += loss.item()
        fpn_train_dice += dice_score(preds, masks)
        fpn_train_iou += iou_score(preds, masks)
        fpn_train_acc += pixel_accuracy(preds, masks)

    # --- Validation ---
    fpn_model.eval()
    fpn_val_loss, fpn_val_dice, fpn_val_iou, fpn_val_acc, fpn_val_prec, fpn_val_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"FPN Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = fpn_model(imgs)
            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            fpn_val_loss += loss.item()
            fpn_val_dice += dice_score(preds, masks)
            fpn_val_iou += iou_score(preds, masks)
            fpn_val_acc += pixel_accuracy(preds, masks)
            fpn_val_prec += precision_score(preds, masks)
            fpn_val_rec += recall_score(preds, masks)

    # --- Test ---
    fpn_test_loss, fpn_test_dice, fpn_test_iou, fpn_test_acc, fpn_test_prec, fpn_test_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(test_loader, desc=f"FPN Epoch {epoch+1}/{num_epochs} - Test"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = fpn_model(imgs)
            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            fpn_test_loss += loss.item()
            fpn_test_dice += dice_score(preds, masks)
            fpn_test_iou += iou_score(preds, masks)
            fpn_test_acc += pixel_accuracy(preds, masks)
            fpn_test_prec += precision_score(preds, masks)
            fpn_test_rec += recall_score(preds, masks)

    # --- Print metrics ---
    print(f"\nFPN Epoch {epoch+1}:")
    print(f" Train Loss={fpn_train_loss/len(train_loader):.4f} | "
          f"Dice={fpn_train_dice/len(train_loader):.4f} | IoU={fpn_train_iou/len(train_loader):.4f} | Acc={fpn_train_acc/len(train_loader):.4f}")
    print(f" Val   Loss={fpn_val_loss/len(val_loader):.4f} | "
          f"Dice={fpn_val_dice/len(val_loader):.4f} | IoU={fpn_val_iou/len(val_loader):.4f} | Acc={fpn_val_acc/len(val_loader):.4f} | "
          f"Prec={fpn_val_prec/len(val_loader):.4f} | Recall={fpn_val_rec/len(val_loader):.4f}")
    print(f" Test  Loss={fpn_test_loss/len(test_loader):.4f} | "
          f"Dice={fpn_test_dice/len(test_loader):.4f} | IoU={fpn_test_iou/len(test_loader):.4f} | Acc={fpn_test_acc/len(test_loader):.4f} | "
          f"Prec={fpn_test_prec/len(test_loader):.4f} | Recall={fpn_test_rec/len(test_loader):.4f}")

    # --- Early stopping ---
    if fpn_test_loss < best_test_loss:
        best_test_loss = fpn_test_loss
        epochs_no_improve = 0
        torch.save(fpn_model.state_dict(), "best_fpn.pth")
        print("✅ FPN Model saved!")
    else:
        epochs_no_improve += 1
        print(f"EarlyStopping counter: {epochs_no_improve} of {patience}")
        if epochs_no_improve >= patience:
            print("⏹️ Early stopping triggered!")
            break

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
FPN Epoch 1/20 - Test: 10


FPN Epoch 1:
 Train Loss=0.5302 | Dice=0.0001 | IoU=0.0000 | Acc=0.9739
 Val   Loss=0.5070 | Dice=0.0000 | IoU=0.0000 | Acc=0.9958 | Prec=0.0000 | Recall=0.0000
 Test  Loss=0.5074 | Dice=0.0000 | IoU=0.0000 | Acc=0.9954 | Prec=0.0000 | Recall=0.0000
✅ FPN Model saved!


FPN Epoch 2/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 24.32it/s]



FPN Epoch 2:
 Train Loss=0.4533 | Dice=0.1060 | IoU=0.0623 | Acc=0.9959
 Val   Loss=0.3656 | Dice=0.2788 | IoU=0.1679 | Acc=0.9964 | Prec=0.8312 | Recall=0.1757
 Test  Loss=0.3983 | Dice=0.2144 | IoU=0.1307 | Acc=0.9960 | Prec=0.7160 | Recall=0.1398
✅ FPN Model saved!


FPN Epoch 3/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 18.67it/s]


...


FPN Epoch 18:
 Train Loss=0.0784 | Dice=0.8462 | IoU=0.7341 | Acc=0.9987
 Val   Loss=0.0933 | Dice=0.8170 | IoU=0.6932 | Acc=0.9985 | Prec=0.7874 | Recall=0.8507
 Test  Loss=0.1160 | Dice=0.7741 | IoU=0.6357 | Acc=0.9980 | Prec=0.7675 | Recall=0.7972
EarlyStopping counter: 1 of 5


FPN Epoch 19/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 21.53it/s]



FPN Epoch 19:
 Train Loss=0.0705 | Dice=0.8618 | IoU=0.7574 | Acc=0.9989
 Val   Loss=0.1002 | Dice=0.8032 | IoU=0.6728 | Acc=0.9986 | Prec=0.9065 | Recall=0.7224
 Test  Loss=0.1290 | Dice=0.7482 | IoU=0.6031 | Acc=0.9981 | Prec=0.8856 | Recall=0.6607
EarlyStopping counter: 2 of 5


FPN Epoch 20/20 - Test: 100%|██████████| 8/8 [00:00<00:00, 24.10it/s]


FPN Epoch 20:
 Train Loss=0.0699 | Dice=0.8627 | IoU=0.7599 | Acc=0.9989
 Val   Loss=0.0970 | Dice=0.8097 | IoU=0.6824 | Acc=0.9986 | Prec=0.8946 | Recall=0.7418
 Test  Loss=0.1249 | Dice=0.7570 | IoU=0.6149 | Acc=0.9981 | Prec=0.8767 | Recall=0.6792
EarlyStopping counter: 3 of 5


In [32]:
print(f"Best Test Loss: {fpn_test_loss/len(test_loader):.4f} at epoch {epoch + 1 - epochs_no_improve}")
print(f"Best Test Dice: {fpn_test_dice/len(test_loader):.4f}")
print(f"Best Test IoU: {fpn_test_iou/len(test_loader):.4f}")
print(f"Best Test Accuracy: {fpn_test_acc/len(test_loader):.4f}")
print(f"Best Test Precision: {fpn_test_prec/len(test_loader):.4f}")
print(f"Best Test Recall: {fpn_test_rec/len(test_loader):.4f}")

Best Test Loss: 0.1249 at epoch 17
Best Test Dice: 0.7570
Best Test IoU: 0.6149
Best Test Accuracy: 0.9981
Best Test Precision: 0.8767
Best Test Recall: 0.6792


In [59]:
fpn_loss = fpn_test_loss/len(test_loader)
fpn_dice = fpn_test_dice/len(test_loader)
fpn_iou = fpn_test_iou/len(test_loader)

In [60]:
# Model ranking based on test_dice (higher is better) and test_iou (higher is better)
# Make sure to divide by the number of test samples to get values in [0, 1]
num_test_samples = len(test_loader.dataset)
model_results = [
    {
        "name": "U-Net",
        "test_loss": u_net_loss,
        "test_dice": u_net_dice,
        "test_iou": u_net_iou
    },
    {
        "name": "DeepLabV3",
        "test_loss": deep_lab_loss,
        "test_dice": deep_lab_dice,
        "test_iou": deep_lab_iou
    },
    {
        "name": "FPN",
        "test_loss": fpn_loss,
        "test_dice": fpn_dice,
        "test_iou": fpn_iou
    }
]

# Rank by Dice (higher is better)
dice_ranking = sorted(model_results, key=lambda x: x["test_dice"], reverse=True)
print("Ranking by Dice Score:")
for rank, result in enumerate(dice_ranking, 1):
    print(f"Rank {rank}: {result['name']} (Dice: {result['test_dice']:.4f})")

print("\nRanking by IoU Score:")
# Rank by IoU (higher is better)
iou_ranking = sorted(model_results, key=lambda x: x["test_iou"], reverse=True)
for rank, result in enumerate(iou_ranking, 1):
    print(f"Rank {rank}: {result['name']} (IoU: {result['test_iou']:.4f})")

Ranking by Dice Score:
Rank 1: U-Net (Dice: 0.8038)
Rank 2: FPN (Dice: 0.7570)
Rank 3: DeepLabV3 (Dice: 0.6656)

Ranking by IoU Score:
Rank 1: U-Net (IoU: 0.6760)
Rank 2: FPN (IoU: 0.6149)
Rank 3: DeepLabV3 (IoU: 0.5171)
